# deeptrack.properties

<a href="https://colab.research.google.com/github/DeepTrackAI/DeepTrack2/blob/develop/tutorials/3-advanced-topics/DTAT306_properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install deeptrack  # Uncomment if running on Colab/Kaggle.

This advanced tutorial introduces the module deeptrack.properties.

## 1. What is a property?

Each feature (instance of the class `Feature`, see [features_example](DTAT301_features.ipynb)) can have several properties (instances of the class `Property`). These properties can be constants, functions, lists, dictionaries, iterators, or slices, providing flexibility in defining and controlling different aspects of the system being modelled.

A propety has a value accessible through the field `current_value`, whose data type is not restricted. 
This value is updated through a sampling rule (method `.update()`), which is passed to the class constructor on initialization. 

In [2]:
import numpy as np

### 1.1. What is a sampling rule?

The sampling rule determines how the value of a property is updated upon calling `.update()`.
A sampling rule is defined when an instance of the class `Property` is created and can be of any type. 
When calling `.update()`, the value of the property is updated according to the first of the following that applies:
    
1.  If the sampling rule has a method `.sample()`, call `.sample()` and return the output.

2.  If the sampling rule is a ``dict``, sample each value and combine the result into a new ``dict`` using the original keys. 

3.  If the sampling rule is a ``list``, sample each element of the list and combine the result into a new ``list``.

4.  If the sampling rule is an iterable, return the next value. If the iterable is exhausted, the value of the property is not changed.

5.  If the sampling rule is callable, call it and return the result.

6.  If none of the above apply, return the sampling rule itself.

In [3]:
from deeptrack.properties import Property, PropertyDict, SequentialProperty

### 1.2. Property with a Constant Value

The simplest example of a property is one that does not change during an update call.
This is commonly either a number or a tuple, but can be any data type that will be evaluated by case 6 above.
If you want to have a constant property with a value that would be evaluated by cases 1-5 (e.g., a list or a function), you can  wrap it as the output of a lambda function.

In [4]:
# Number.

P = Property(1)
print(f"The current value of the property is {P()}")

P.update()  # Numbers are not changed after an update() call.
print(f"The current value of the property is {P()}")

The current value of the property is 1
The current value of the property is 1


In [5]:
# Tuple.

P = Property((1, [2, 3], None))
print(f"The current value of the property is {P()}")

P.update()  # Tuples are not changed after an update() call.
print(f"The current value of the property is {P()}")

The current value of the property is (1, [2, 3], None)
The current value of the property is (1, [2, 3], None)


In [6]:
# Wrapped list.

P = Property(lambda: [np.random.rand, 1, {}])
print(f"The current value of the property is {P()}")

P.update()  # Objects wrapped in functions are not changed by an update() call.
print(f"The current value of the property is {P()}")

The current value of the property is [<built-in method rand of numpy.random.mtrand.RandomState object at 0x10b30ed40>, 1, {}]
The current value of the property is [<built-in method rand of numpy.random.mtrand.RandomState object at 0x10b30ed40>, 1, {}]


### 1.3. Property with a Discrete Random Value 

Discrete randomness can be achieved by a function (case 5).

In [7]:
# Function.

P = Property(lambda: np.random.randint(0, 10))

for _ in range(5): 
    P.update()
    print(f"The current value of the property is {P()}")

The current value of the property is 9
The current value of the property is 0
The current value of the property is 0
The current value of the property is 2
The current value of the property is 3


In [8]:
# Binary choice.

P = Property(lambda: 1 if np.random.rand() > 0.75 else 0)

for _ in range(5): 
    P.update()
    print(f"The current value of the property is {P()}")

The current value of the property is 0
The current value of the property is 0
The current value of the property is 0
The current value of the property is 1
The current value of the property is 0


### 1.4. Property with a Continuous Random Value

Continuous randomness is typically achieved by passing a function that returns a continuous random value. This function should take no input, as noted in case 5. To use a function that needs arguments, wrap it in a function that calls it with the correct arguments.

In [9]:
# Function with no input.

P = Property(np.random.rand)

for _ in range(5):
    P.update()
    print(f"The current value of the property is {P()}")

The current value of the property is 0.6058058486216974
The current value of the property is 0.4388130582114753
The current value of the property is 0.4492343759700439
The current value of the property is 0.43076786487677377
The current value of the property is 0.5294829032012791


In [10]:
# Wrapped function.

P = Property(lambda: np.random.normal(1, 5))

for _ in range(5):
    P.update()
    print(f"The current value of the property is {P()}")

The current value of the property is -2.7800167909302074
The current value of the property is -4.99826898249571
The current value of the property is 9.239257264425211
The current value of the property is 10.638607727647866
The current value of the property is 7.443379615451991


### 1.5. Property with a Deterministically Changing Value

Deterministically changing properties can be achieved using either an iterator (case 4) or a function (case 5). For the output of a function to change deterministically between calls, it should reference some variable outside its definition. Once an iterator has been exhausted, it will always return its last value.

In [11]:
# Iterator.

P = Property(iter([1, 2, 3, 4, 5]))

for _ in range(10):
    P.update()
    print(f"The current value of the property is {P()}")

The current value of the property is 1
The current value of the property is 2
The current value of the property is 3
The current value of the property is 4
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5


In [12]:
# Function.

fibbonacci = [1, 1]

def fibbonacci_sequence():
    fibbonacci.append(fibbonacci[-2] + fibbonacci[-1])
    return fibbonacci

P = Property(fibbonacci_sequence)

for _ in range(10):
    P.update()
    print(f"The current value of the property is {P()}")

The current value of the property is [1, 1, 2]
The current value of the property is [1, 1, 2, 3]
The current value of the property is [1, 1, 2, 3, 5]
The current value of the property is [1, 1, 2, 3, 5, 8]
The current value of the property is [1, 1, 2, 3, 5, 8, 13]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]


### 1.6. Property with Dependent Value

The value of a property can be dependent on the value on some other property. It does this by accepting some keyword argument corresponding to the name of the independent property. Instances of `Feature` will handle this automatically.

In [13]:
random_number = Property(lambda: np.random.rand())

def get_dependent_number():
    return random_number() + 1

dependent_number = Property(get_dependent_number)

# Link the properties with add_dependency() or add_child().
# PropertyDict (see below) automatically links the properties.
dependent_number.add_dependency(random_number)  # Alternative 1.
# random_number.add_child(dependent_number)  # Alternative 2.

for _ in range(5):
    dependent_number.update()
    
    print(f"The current independent property is {random_number()}")
    print(f"The current dependent property is {dependent_number()}\n")
    


The current independent property is 0.6047967671299797
The current dependent property is 1.6047967671299797

The current independent property is 0.9632829097510287
The current dependent property is 1.9632829097510287

The current independent property is 0.5707308224509307
The current dependent property is 1.5707308224509307

The current independent property is 0.17336784973310793
The current dependent property is 1.173367849733108

The current independent property is 0.2129111133514252
The current dependent property is 1.2129111133514252



## 3. What is a PropertyDict?

Another class contained in the module deeptrack.properties is `PropertyDict`. This is a dictionary of properties (keys: name of properties; values: properties) complemented by utility methods to manage collections of properties. These include:

- `.current_value_dict()`, which creates and returns a dictionary with the current value of all properties in the PropertyDict (keys: name of properties; values: current values of the properties).

- `.update()`, which calls the method `.update()` on all properties in the PropertyDict.

- `.sample()`, which calls the method `.sample()` on all properties in the PropertyDict, and creates and returns a dictionary from the output (keys: name of properties; values: sample outputs of the properties).

Importantly, `PropertyDict` automatically links the properties within a property dictionary.


In [14]:
property_dict = PropertyDict(
    foo="foo",
    barorbaz=lambda:np.random.choice(["bar", "baz"]),
    foobarorbaz=lambda foo, barorbaz: foo + barorbaz,
)

for _ in range(5):
    property_dict.update()
    print(f"The current properties in property_dict are {property_dict()}")

The current properties in property_dict are {'foo': 'foo', 'barorbaz': 'baz', 'foobarorbaz': 'foobaz'}
The current properties in property_dict are {'foo': 'foo', 'barorbaz': 'baz', 'foobarorbaz': 'foobaz'}
The current properties in property_dict are {'foo': 'foo', 'barorbaz': 'bar', 'foobarorbaz': 'foobar'}
The current properties in property_dict are {'foo': 'foo', 'barorbaz': 'bar', 'foobarorbaz': 'foobar'}
The current properties in property_dict are {'foo': 'foo', 'barorbaz': 'baz', 'foobarorbaz': 'foobaz'}
